In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

'3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:55:37) \n[Clang 14.0.6 ]'

In [3]:
%load_ext autoreload 
%autoreload 2

In [4]:
UNIQUE_CNT = 10
VERSION = "v1"

### Read data

In [5]:
train_test = pd.read_csv("../../data/train_validation_marker.csv")
train_test["SALES_DATE"] = pd.to_datetime(train_test["SALES_DATE"])
print (train_test.shape)

(846632, 15)


#### Find SKUs with very low volume

In [6]:
#clean train/test
train = train_test[(train_test.validation==False) & (train_test.validation_clean==True)]
validation = train_test[(train_test.validation==True) & (train_test.validation_clean==True)]
train.shape,validation.shape

((797490, 15), (3815, 15))

In [7]:
sku_vals = train[train.SALES_DATE>="2022-03-01"].groupby("Encoded_SKU_ID")["DAILY_UNITS"].nunique()

In [8]:
(sku_vals<=UNIQUE_CNT).sum()

364

In [9]:
sku_vals.count()

545

### Fit HMM on these

In [10]:
sku_in_scope = sku_vals[(sku_vals<=UNIQUE_CNT)]

In [11]:
import sys
sys.path.append("../../")

from src.ad_hmm import sku_predict

In [12]:
pd.options.mode.chained_assignment = None

In [13]:
from tqdm.notebook import tqdm

In [14]:
sku_in_scope.index[:1]

Int64Index([1], dtype='int64', name='Encoded_SKU_ID')

In [15]:
from IPython.utils import io

In [ ]:
all_preds = []
for sku_id in tqdm(sku_in_scope.index):
    n_comps = sku_in_scope[sku_id]
    sku_pred_model = sku_predict(train_test,sku_id)
    try:
#         sku_pred_15 = sku_pred_model.predict(valid1, 15)
#         sku_pred_30 = sku_pred_model.predict(valid1, 30)
#         sku_pred_45 = sku_pred_model.predict(valid1, 45)
#         sku_pred_60 = sku_pred_model.predict(valid1, 60)
        with io.capture_output() as captured:
            feats_sku = sku_pred_model.get_features(n_lags=2)
            train1,valid1 = sku_pred_model.split_train_test("2022-07-25")
            sku_pred_model.fit_hmm(train1,"2022-03-01", n_components1 = n_comps)
            sku_pred_90 = sku_pred_model.predict(valid1, 90)
            sku_pred_max = sku_pred_model.predict(valid1, sku_pred_model.X.shape[0])
        merge0 = sku_pred_90[["predicted"]].merge(sku_pred_max[["predicted"]],left_index = True,\
                              right_index = True, suffixes=("","_max"))
#         merge1 = merge0.merge(sku_pred_45[["predicted"]],left_index = True,\
#                               right_index = True, suffixes=("","_45"))
#         merge2 = merge1.merge(sku_pred_60[["predicted"]],left_index = True,\
#                               right_index = True, suffixes=("","_60"))
#         merge3 = merge2.merge(sku_pred_90[["predicted"]],left_index = True,\
#                               right_index = True, suffixes=("","_90"))
#         merge4 = merge2.merge(sku_pred_max[["predicted"]],left_index = True,\
#                               right_index = True, suffixes=("","_max"))
        merge0["Encoded_SKU_ID"] =[sku_id]*merge0.shape[0]
        all_preds.append(merge0)
    except:
        print (f"Error for {sku_id}")

  0%|          | 0/364 [00:00<?, ?it/s]

Error for 60
Error for 69
Error for 198
Error for 199
Error for 202
Error for 203
Error for 213
Error for 214
Error for 215
Error for 216
Error for 219
Error for 221
Error for 222
Error for 223
Error for 224
Error for 225
Error for 226
Error for 227
Error for 228
Error for 229
Error for 230
Error for 234
Error for 239
Error for 243
Error for 245
Error for 246
Error for 247
Error for 248
Error for 249
Error for 251
Error for 254
Error for 256
Error for 258
Error for 259
Error for 260
Error for 261
Error for 262
Error for 264
Error for 266
Error for 268
Error for 272
Error for 275
Error for 276
Error for 277
Error for 280
Error for 281
Error for 282
Error for 283
Error for 284
Error for 288
Error for 289
Error for 291
Error for 292
Error for 293
Error for 294
Error for 295
Error for 296
Error for 297
Error for 298
Error for 300
Error for 302
Error for 304
Error for 305
Error for 306
Error for 308
Error for 309
Error for 310
Error for 311
Error for 313
Error for 314
Error for 316
Error fo

In [ ]:
hmm_result = pd.concat(all_preds)
hmm_result.to_csv(f"../../data/hmm_result_{VERSION}.csv")